### Here we include the **conda** installed library using Cppyy

In [1]:
!conda install -y -c conda-forge eigen

Solving environment: ...working... done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.7.4

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

Retrieving notices: ...working... done


In [2]:
#include <clang/Interpreter/CppInterOp.h>
Cpp::AddIncludePath("/opt/conda/envs/.venv/include");

### We are immediately able to use the `Eigen::Dense` namespaces in our C++ code cell

In [3]:
#include "/opt/conda/envs/.venv/include/eigen3/Eigen/Dense"

typedef Eigen::Matrix<double, Eigen::Dynamic, Eigen::Dynamic> MatrixXd;

Eigen::Vector2d a;
Eigen::Vector3d b;
Eigen::Vector4d c;

### We can define a templated class with a set of vector operations offered by Eigen 

In [4]:
template <typename VectorType>
class EigenOperations {
public:
    static VectorType PerformOperations(const VectorType& v) {
        VectorType result = v;
        
        // Using Eigen::Dense object methods
        
        result.normalize(); // eigen vector normalisation

        double dot = v.dot(v); // dot product
        for (int i = 0; i < result.size(); i++) {
            result[i] += dot;
        }
 
        double squaredNorm = v.squaredNorm(); // vector squared norm 
        for (int i = 0; i < result.size(); i++) {
            result[i] -= squaredNorm;
        }

        return result;
    }
};


### Accessing the declared Eigen vectors on the Python side using Cppyy

In [5]:
%%python

import cppyy

a = cppyy.gbl.a
b = cppyy.gbl.b
c = cppyy.gbl.c

def display_eigen(vec, name):
    print("Vector : ", name, ", Dimensions : ", len(vec))
    for x in vec:
        print(x)

#### Setting values to the C++ Eigen Vector

In [6]:
%%python

for i in range(len(a)):
    a[i] = i - 1

for i in range(len(b)):
    b[i] = i + 1

for i in range(len(c)):
    c[i] = i - 1

In [7]:
%%python

display_eigen(a, "A")
display_eigen(b, "B")
display_eigen(c, "C")

Vector :  A , Dimensions :  2
-1.0
0.0
Vector :  B , Dimensions :  3
1.0
2.0
3.0
Vector :  C , Dimensions :  4
-1.0
0.0
1.0
2.0


### Calling the Templated method of the C++ class from the Python side

In [8]:
%%python

res2d = cppyy.gbl.EigenOperations["Eigen::Vector2d"].PerformOperations(a)
res3d = cppyy.gbl.EigenOperations["Eigen::Vector3d"].PerformOperations(b)
res4d = cppyy.gbl.EigenOperations["Eigen::Vector4d"].PerformOperations(c)

In [9]:
!pip install matplotlib

In [10]:
%%python

import matplotlib.pyplot as plt
import numpy as np

origin = [0, 0]

np_res2 = np.array(list(res2d))
np_res3 = np.array(list(res3d)) 
np_res4 = np.array(list(res4d)) 

plt.quiver(*origin, *np_res2, color=['r'], scale=5)
plt.quiver(*origin, *np_res3, color=['b'], scale=5)


plt.savefig("test.jpg")
plt.show()

<img src="./test.jpg"/>

In [12]:
%%python

display_eigen(res2d, "A")
display_eigen(res3d, "B")
display_eigen(res4d, "C")

Vector :  A , Dimensions :  2
-1.0
0.0
Vector :  B , Dimensions :  3
0.26726124191242384
0.5345224838248495
0.8017837257372733
Vector :  C , Dimensions :  4
-0.40824829046386313
0.0
0.40824829046386313
0.8164965809277263
